# Example notebook of how analysis can be scaled up

In [ ]:
import sys
import os
sys.path.append('/home/jovyan/intake-aodn')
import intake_aodn
import intake

## Request a cluster with 2 CPU per worker, 4GB ram per worker, and allow to scale up to 64 workers

In [ ]:
from intake_aodn.utils import get_distributed_cluster
client = get_distributed_cluster(worker_cores=2,
                                 worker_memory=4,
                                 min_workers=1,
                                 max_workers=64)
client

In [ ]:
# Load credentials
def load_creds():
    with open(os.environ['HOME'] + '/.aws/credentials','rt') as f:
        f.readline()
        key=f.readline().split('=')[1].strip()
        secret=f.readline().split('=')[1].strip()
    return key, secret

key,secret=load_creds()

storage_options=dict(target_protocol='s3',
                     target_options=dict(key=key,secret=secret),
                     remote_protocol='s3',
                     remote_options=dict(anon=True))

In [ ]:
cat=intake_aodn.cat

# Open 20 years of MODIS Chl-a across all of WA

In [ ]:
ds=cat.aodn_s3.MODIS_1d_chl_oc3(startdt='2002-01-01',
                                enddt='2021-06-30',
                                storage_options=storage_options,
                                cropto=dict(longitude=slice(107.,128),
                                            latitude=slice(-12,-37))).to_dask()

In [ ]:
ds

In [ ]:
print(f'{ds.nbytes/1E9} GB')

## "Persist" the data from S3 into cluster memory for interactive viewing

In [ ]:
ds = ds.persist()

In [ ]:
import hvplot.xarray
import geoviews as gv
from cartopy import crs as ccrs
gv.extension('bokeh')

In [ ]:
%%opts Overlay [width=900 height=600]

import warnings
warnings.filterwarnings("ignore")

(ds.chl_oc3.hvplot('longitude','latitude',groupby='time', dynamic=True, rasterize=True, cmap='magma',crs=ccrs.PlateCarree()).redim(chl_oc3=dict(range=(0, 1.0))))*\
(gv.feature.coastline(scale='10m')).opts(color='black')

# WA wide monthly mean

In [ ]:
ds_mean = ds.chl_oc3.groupby('time.month').mean()

In [ ]:
print(f'{ds_mean.nbytes/1E9} GB')

In [ ]:
ds_mean = ds_mean.load()

In [ ]:
%%opts Overlay [width=900 height=600]

import warnings
warnings.filterwarnings("ignore")

(ds_mean.hvplot('longitude','latitude',groupby='month',dynamic=True, rasterize=True, cmap='magma',crs=ccrs.PlateCarree()).redim(chl_oc3=dict(range=(0, 5.0))))*\
(gv.feature.coastline(scale='10m')).opts(color='black')

In [ ]:
client.cluster.shutdown()